In [15]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
wandb_log_name = "CC_HC_GOM_30_platf_coll_100m_flocking_0_5"
wandb_log = False

### Define Filenames

+ Hindcast-Hindcast

In [19]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = '30_platforms_HC_HC_col_100m_hj_naive_1181_fiedler.csv'
pb_name_reactive = '30_platforms_HC_HC_col_100m_reactive_ctrl_1176_fiedler.csv'
pb_name_flocking = '30_platforms_HC_HC_col_100m_flocking_standard_1_1106.csv'#30_platforms_HC_HC_col_100m_flocking_1_1179.csv' #'30_platforms_HC_HC_col_100m_flocking_0_5_1120.csv'#'30_platforms_HC_HC_col_100m_flocking_0_1_1193.csv' #'30_platforms_HC_HC_flocking_0_1_1179.csv'

+ Forecast-Hindcast

In [7]:
# path_to_files = 'generated_media/metrics_csv_results/'
# #file_containing_pb = 'problemsGOM.csv'
# pb_name_hj_naive = '30_platforms_FC_HC_col_100m_hj_naive_1122_fiedler.csv'
# pb_name_reactive = '30_platforms_FC_HC_col_100m_reactive_ctrl_1126_fiedler.csv'
# pb_name_flocking = '30_platforms_FC_HC_col_100m_flocking_0_5_1121_fiedler.csv'

### Plot problem distribution

In [ ]:
# TODO

### Extract metrics from csv files

In [20]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [21]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [22]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [23]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [24]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  105 corresponds to 9.704251386321626 %


In [25]:
metrics_hj_naive

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mean solver time in seconds,std solver time in seconds,fiedler_mean,fiedler_min,fiedler_max,nb_disconnections
mission_nr,,,,,,,,,,,,,,,,
0,0,0.000000,158,1.000000,0.000000,0.0,3.952678,11,9,0,2.812443,0.669795,1.760829,6.471318e-02,12.487573,0
1,1,0.733410,0,1.000000,0.000000,0.0,5.530790,9,25,0,2.944291,0.661673,1.915889,6.097773e-01,4.848381,0
3,417,2.090278,727,0.933333,0.006699,0.0,3.401464,15,15,0,2.929138,0.643447,0.250279,-1.188953e-15,2.302795,144
4,528,0.231481,55,1.000000,0.000000,0.0,3.704008,12,12,0,2.880107,0.645540,1.206489,-1.776357e-15,4.918294,0
5,638,1.993441,0,1.000000,0.000000,0.0,2.070713,11,8,0,2.860497,0.693655,0.022562,-2.598632e-15,0.787185,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,216,0.793210,101,0.900000,0.000988,0.0,4.123083,10,17,0,2.982834,0.670251,0.089348,-2.832337e-15,0.926582,57
1196,217,0.745370,1523,1.000000,0.000000,0.0,6.776378,13,22,0,2.877588,0.665752,8.746128,0.000000e+00,26.000000,24
1197,218,0.000000,1149,1.000000,0.000000,0.0,7.649133,12,29,0,2.899481,0.659521,11.358049,4.080695e+00,30.000000,0


+ Drop infeasible missions on HC

In [26]:
metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [27]:
import plotly.express as px
import plotly.graph_objects as go

In [28]:
print("number of missions considered: ", len(metrics_hj_naive))

number of missions considered:  977


In [29]:
metrics_hj_naive['fiedler_min'].mean()

0.3892580934010998

In [31]:
metrics_hj_reactive['fiedler_min'].mean()

0.41718332222172705

In [30]:
metrics_hj_flocking['fiedler_min'].mean()

1.1218738785579814

In [32]:
metrics_hj_naive[metrics_hj_naive['Isolated_platform_metric']>0]

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mean solver time in seconds,std solver time in seconds,fiedler_mean,fiedler_min,fiedler_max,nb_disconnections
mission_nr,,,,,,,,,,,,,,,,
1,1,0.733410,0,1.0,0.0,0.0,5.530790,9,25,0,2.944291,0.661673,1.915889,6.097773e-01,4.848381,0
4,528,0.231481,55,1.0,0.0,0.0,3.704008,12,12,0,2.880107,0.645540,1.206489,-1.776357e-15,4.918294,0
5,638,1.993441,0,1.0,0.0,0.0,2.070713,11,8,0,2.860497,0.693655,0.022562,-2.598632e-15,0.787185,4
6,747,0.307870,950,1.0,0.0,0.0,6.411098,8,28,0,2.857606,0.652185,0.925271,0.000000e+00,3.184823,6
8,963,0.369599,0,1.0,0.0,0.0,5.212678,9,22,0,2.172468,0.522170,1.395619,0.000000e+00,13.248081,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,204,0.864969,0,1.0,0.0,0.0,3.198844,10,21,0,2.864217,0.710928,0.575893,-1.264560e-15,8.481359,58
1187,207,0.141204,50,1.0,0.0,0.0,4.428131,11,10,0,2.952975,0.639347,0.604108,7.652225e-02,1.443472,0
1188,208,0.053241,267,1.0,0.0,0.0,5.901002,9,20,0,2.824381,0.679996,3.336539,0.000000e+00,8.640276,7


## Box Plots

In [33]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Mean Fiedler value 

In [34]:
fiedler_dict = {"hj_naive": metrics_hj_naive["fiedler_mean"],
                      "hj_reactive": metrics_hj_reactive["fiedler_mean"],
                      "flocking": metrics_hj_flocking["fiedler_mean"]}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Fiedler value over Missions"
)
fig_beta.show()

In [35]:
fiedler_dict = {"hj_naive": np.abs(metrics_hj_naive["fiedler_min"]),
                      "hj_reactive": np.abs(metrics_hj_reactive["fiedler_min"]),
                      "flocking": np.abs(metrics_hj_flocking["fiedler_min"])}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
#Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    height=800,
    width=1000
)
# fig_beta.update_layout(
#     xaxis_title="controllers",
#     yaxis_title="Minimum Fiedler value over Missions",
#     height=800,
#     width=1000,
#     yaxis=dict(
#         type='log',
#         #dtick=0.2,
#         tickvals=[1,2],
#         ticktext=['0','1' ],
#         range=[0,2]
#         # showexponent='none'
#     )
# )
fig_beta.show()

+ Min Fiedler value 

In [27]:
fiedler_dict = {"hj_naive": np.abs(metrics_hj_naive["fiedler_min"])+1,
                      "hj_reactive": np.abs(metrics_hj_reactive["fiedler_min"])+1,
                      "flocking": np.abs(metrics_hj_flocking["fiedler_min"])+1}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all", log_y=True)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
# fig_beta.update_layout(
#     xaxis_title="controllers",
#     yaxis_title="Minimum Fiedler value over Missions",
#     height=800,
#     width=1000
# )
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    height=800,
    width=1000,
    yaxis=dict(
        type='log',
        #dtick=0.2,
        tickvals=[1,10*np.log10(2+1), 10*np.log10(5+1)],
        ticktext=['0','2', '5' ],
        range=[0,1]
        # showexponent='none'
    )
)
fig_beta.show()

In [167]:
np.log(min(fiedler_dict["hj_naive"])+1e-15)

-34.538776394910684

In [85]:
import plotly.express as px
import pandas as pd

# Create data frame
fiedler_dict = {"hj_naive": metrics_hj_naive["fiedler_min"],
                "hj_reactive": metrics_hj_reactive["fiedler_min"],
                "flocking": metrics_hj_flocking["fiedler_min"]}
beta_index_avg_df = pd.DataFrame(fiedler_dict)

# Create a list of colors based on the column values
colors = []
for col in beta_index_avg_df.columns:
    colors += ['red' if x < 1 else 'blue' for x in beta_index_avg_df[col]]
    # colors += ['red' if x < 1 else 'red' for x in beta_index_avg_df[col]]

# Create violin plot
fig_beta = px.box(beta_index_avg_df.melt(var_name='Controller', value_name='Minimum Fiedler value over Missions'), x='Controller', y='Minimum Fiedler value over Missions',
                      boxmode='overlay',color=colors, points="all")

# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="Controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    legend_traceorder='reversed',
    height=800,
    width=1000
)
# Update the legend labels
fig_beta.update_traces(
    name='below 1',
    selector={'legendgroup': 'red'}
)
fig_beta.update_traces(
    name='above 1',
    selector={'legendgroup': 'blue'}
)

fig_beta.show()


In [45]:
beta_index_avg_df['hj_naive']

mission_nr
0       0.000000e+00
1       7.753889e-01
2      -1.632120e-15
3      -1.642506e-15
5      -8.272148e-16
            ...     
1193   -1.492569e-15
1196    1.140877e+00
1197    1.019299e-04
1198    1.547686e+00
1199    6.638187e-01
Name: hj_naive, Length: 1014, dtype: float64

+ Beta Index

In [13]:
beta_index_avg_dic = {"hj_naive": metrics_hj_naive["Average Beta Index"],
                      "hj_reactive": metrics_hj_reactive["Average Beta Index"],
                      "flocking": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.violin(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Beta Index over Missions"
)
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [36]:
mean_min_dist_dict = {"hj_naive": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "hj_reactive": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "flocking":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}
idx_list = []
mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.violin(mean_min_dist_df, log_y=True)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="controllers",
    yaxis_title=" [km]",
    title = {'text': "Mean minimum distance to target among all platforms"}
)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [37]:
opt_ctrl_correction_dict = {"hj_naive": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "hj_reactive": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "flocking": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title="controllers",
    yaxis_title="degrees",
    title = {'text': "Control direction correction from optimal input"}
)
fig_opt_ctrl_correction.show()

+ Isolated Platform Metric

In [38]:

failed_connectivity_hj_flocking =  metrics_hj_flocking#metrics_hj_flocking[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_naive = metrics_hj_naive#metrics_hj_naive[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_reactive = metrics_hj_reactive #metrics_hj_reactive[metrics_hj_flocking["Isolated_platform_metric"]>0]


ipm_dict= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"}
)
fig_ipm.show()

In [39]:
failed_connectivity_hj_flocking =  metrics_hj_flocking#metrics_hj_flocking[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_naive = metrics_hj_naive#metrics_hj_naive[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_reactive = metrics_hj_reactive #metrics_hj_reactive[metrics_hj_flocking["Isolated_platform_metric"]>0]


ipm_dict= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"},
    height=800,
    width=1000
)
fig_ipm.show()

In [40]:
# extract missions for which flocking failed to maintain connectivity
failed_idx_flocking = metrics_hj_flocking["Isolated_platform_metric"]>0

failed_hj_flocking =  metrics_hj_flocking[failed_idx_flocking]
failed_hj_naive = metrics_hj_naive[failed_idx_flocking]
failed_hj_reactive = metrics_hj_reactive[failed_idx_flocking]

ipm_dict_all= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking":metrics_hj_flocking["Isolated_platform_metric"]}

ipm_dict_failed_connect= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}

all_x_ticks = ['hj_naive']*len(metrics_hj_naive) + ['hj_reactive']*len(metrics_hj_reactive) + ['flocking']*len(metrics_hj_flocking)
all_missions = list(metrics_hj_naive["Isolated_platform_metric"])+list(metrics_hj_reactive["Isolated_platform_metric"])+ list(metrics_hj_flocking["Isolated_platform_metric"])
failed_x_ticks = ['hj_naive']*len(failed_hj_naive) + ['hj_reactive']*len(failed_hj_reactive) + ['flocking']*len(failed_hj_flocking)
failed_missions = list(failed_hj_naive["Isolated_platform_metric"])+list(failed_hj_reactive["Isolated_platform_metric"])+ list(failed_hj_flocking["Isolated_platform_metric"])
fig = go.Figure()
# HJ naive
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color='#636EFA'), name = 'all missions'))
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color='#EF553B'), name = 'failed missions only'))
fig.update_layout(
    yaxis_title='IPM',
    boxmode='group', # group together boxes of the different traces for each value of x
    height=800,
    width=1000
)

fig.show()

In [120]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1,horizontal_spacing= 0.075)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='IPM', row=1, col=2)

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=800, width=1200)

# Show the plot
fig.show()

In [34]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# failed_idx_flocking = metrics_hj_flocking["nb_disconnections"]>0
failed_hj_flocking =  metrics_hj_flocking[(metrics_hj_flocking["nb_disconnections"]>0) & (metrics_hj_flocking["Isolated_platform_metric"]>0)]
failed_hj_naive = metrics_hj_naive[(metrics_hj_naive["nb_disconnections"]>0) & (metrics_hj_naive["Isolated_platform_metric"]>0)]
failed_hj_reactive = metrics_hj_reactive[(metrics_hj_reactive["nb_disconnections"]>0) & (metrics_hj_reactive["Isolated_platform_metric"]>0)]

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1,horizontal_spacing= 0.1)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

all_x_ticks = ['hj_naive']*len(metrics_hj_naive) + ['hj_reactive']*len(metrics_hj_reactive) + ['flocking']*len(metrics_hj_flocking)
all_missions = list(metrics_hj_naive["Isolated_platform_metric"])+list(metrics_hj_reactive["Isolated_platform_metric"])+ list(metrics_hj_flocking["Isolated_platform_metric"])
failed_x_ticks = ['hj_naive']*len(failed_hj_naive) + ['hj_reactive']*len(failed_hj_reactive) + ['flocking']*len(failed_hj_flocking)
failed_missions = list(failed_hj_naive["Isolated_platform_metric"]/failed_hj_naive['nb_disconnections'])+list(failed_hj_reactive["Isolated_platform_metric"]/failed_hj_reactive['nb_disconnections'])+ \
                  list(failed_hj_flocking["Isolated_platform_metric"]/failed_hj_flocking['nb_disconnections'])

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=[miss*24*6 for miss in failed_missions], jitter=0.25, pointpos=+1.6, boxpoints= 'all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='Disconnection durations', row=1, col=2,type='log' ) #range=[0, 100] ?

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=800, width=1200)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1, horizontal_spacing=0.075)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='IPM', row=1, col=2)

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=600, width=1200)

# Set the legend for each subplot
# Set the legend for each subplot
fig.update_layout(
    legend=dict(
        traceorder='normal',
        font=dict(size=12),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2,
        orientation='v',
        yanchor='top',
        xanchor='left',
        y=1.0,
        x=0.01
    ),
    legend_tracegroupgap=20,
    # Set the blue legend to only show the "All Missions" item
    legend_items=[dict(label='All Missions', color=all_missions_color)]
)

fig.update_layout(
    legend=dict(
        traceorder='normal',
        font=dict(size=12),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2,
        orientation='v',
        yanchor='top',
        xanchor='left',
        y=1.0,
        x=0.55
    ),
    legend_tracegroupgap=20,
    # Set the red legend to only show the "Failed Missions Only" item
    legend_items=[dict(label='Failed Missions Only', color=failed_missions_color)]
)


# Show the plot
fig.show()


In [29]:
metrics_hj_naive.mean()

Unnamed: 0                                                    576.027613
Isolated_platform_metric                                        0.947355
Number_of_collision                                           135.183432
Reaching_target                                                 0.577844
Mean minimum distance to target among all platforms in deg      0.097387
Mean maximum correction from optimal control degrees            0.000000
Average Beta Index                                              3.638457
Initial maximum degree of the graph                             9.858974
Final maximum degree of the graph                              14.344181
Mission_success                                                 0.016765
mean solver time in seconds                                     2.773936
std solver time in seconds                                      3.242042
fiedler_mean                                                    2.046511
fiedler_min                                        

### Get Metrics summary as a dict:

In [41]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "std_beta_index":metrics_file["Average Beta Index"].std(),
    "mean_of_min_fiedler_value":metrics_file["fiedler_min"].abs().mean(),
    "std_of_min_fiedler_value": metrics_file["fiedler_min"].abs().std(),
    "avg_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].mean(),
    "std_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].std(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "std_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].std(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "std_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].std()).km,
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "rate_of_missions_with_disconnection":np.mean(metrics_file["Isolated_platform_metric"] > 0), 
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "rate_of_missions_with_collisions": np.mean(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [42]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
print(metrics_summary_hj_naive)
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])

{'ctrl_name': 'hj_naive', 'avg_beta_index': 4.926254765186969, 'std_beta_index': 1.9110605370594838, 'mean_of_min_fiedler_value': 0.3892580934011011, 'std_of_min_fiedler_value': 0.6353996654544469, 'avg_isolated_pltf_integral': 0.3731223226809204, 'std_isolated_pltf_integral': 0.5181757305678425, 'avg_correction_from_opt_ctrl_deg': 0.0, 'std_correction_from_opt_ctrl_deg': 0.0, 'avg_mean_min_dist_to_target_km': 0.0, 'std_mean_min_dist_to_target_km': 0.0, 'nb_missions_with_isolated_pltf': 625, 'rate_of_missions_with_disconnection': 0.6397134083930399, 'nb_missions_with_collisions': 670, 'rate_of_missions_with_collisions': 0.6857727737973388, 'nb_missions_success': 78}


+ HJ Multi-Agent Decentralized Reactive Control

In [43]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
print(metrics_summary_hj_reactive)
summary_hj_naive_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])

{'ctrl_name': 'hj_reactive', 'avg_beta_index': 4.646331757592252, 'std_beta_index': 1.5718851452241496, 'mean_of_min_fiedler_value': 0.41718332222172844, 'std_of_min_fiedler_value': 0.6234787854027195, 'avg_isolated_pltf_integral': 0.1909726171070422, 'std_isolated_pltf_integral': 0.32950551519084414, 'avg_correction_from_opt_ctrl_deg': 81.59617015897078, 'std_correction_from_opt_ctrl_deg': 25.31269733556498, 'avg_mean_min_dist_to_target_km': 0.1432026407533385, 'std_mean_min_dist_to_target_km': 0.5702648666659905, 'nb_missions_with_isolated_pltf': 471, 'rate_of_missions_with_disconnection': 0.4820880245649949, 'nb_missions_with_collisions': 0, 'rate_of_missions_with_collisions': 0.0, 'nb_missions_success': 395}


+ HJ Multi-Agent Flocking

In [45]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
print(metrics_summary_hj_flocking)
summary_hj_naive_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])

{'ctrl_name': 'hj_flocking', 'avg_beta_index': 11.690849184420866, 'std_beta_index': 1.2680294855814318, 'mean_of_min_fiedler_value': 1.1218738785579814, 'std_of_min_fiedler_value': 1.0314305649815834, 'avg_isolated_pltf_integral': 0.044426121788796645, 'std_isolated_pltf_integral': 0.1818676945498458, 'avg_correction_from_opt_ctrl_deg': 133.5287594700594, 'std_correction_from_opt_ctrl_deg': 20.710820232321073, 'avg_mean_min_dist_to_target_km': 4.817901247750761, 'std_mean_min_dist_to_target_km': 11.915605517199765, 'nb_missions_with_isolated_pltf': 91, 'rate_of_missions_with_disconnection': 0.0931422722620266, 'nb_missions_with_collisions': 14, 'rate_of_missions_with_collisions': 0.014329580348004094, 'nb_missions_success': 481}


## Confusion Matrices Plots

In [ ]:
def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label], normalize=True)*100
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_traces(texttemplate='%{text:.1f}%', textfont_size=12, text=confusion_matrix.values)
    #fig = px.imshow(confusion_matrix.round(2), text_auto=True)
    fig.update_layout(
        title = {'text': title_name}
    )
    fig.show()
    # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, fmt='g')
    # # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, norm=LogNorm(), fmt='g') # if heat map is in norm
    # plt.tick_params(axis='both', which='major', labelbottom=False, bottom=False, top=True, labeltop=True)
    # ax.xaxis.set_label_position('top')
    # ax.plot
    # plt.title(title_name)
    # plt.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [ ]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_reactive["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [ ]:
connect_objective_reactive = (metrics_hj_reactive["Isolated_platform_metric"]==0) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Reactive Control', title_name= 'Missions without collisions and with maintained connectivity')

+  Missions success counts

In [ ]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [ ]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_flocking["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [ ]:
connect_objective_flocking = (metrics_hj_flocking["Isolated_platform_metric"]==0) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Flocking Control', title_name= 'Missions without collisions and with maintained connectivity')

In [ ]:
index_list = metrics_hj_flocking.index[(connect_objective_reactive==True) & (connect_objective_flocking==False)].tolist()


In [ ]:
index_list

In [ ]:
metrics_hj_reactive.loc[285]

+  Missions success counts

In [ ]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

# Statistical Testing

In [41]:
import numpy as np
from scipy.stats import ttest_ind

reactive_X = list(metrics_hj_reactive['Average Beta Index']) 
flocking_X = list(metrics_hj_flocking['Average Beta Index'])

# reactive_X = list(metrics_hj_reactive['Isolated_platform_metric']) 
# flocking_X= list(metrics_hj_flocking['Isolated_platform_metric'])
reactive_X = list(abs(metrics_hj_reactive['fiedler_min']))
flocking_X = list(abs(metrics_hj_flocking['fiedler_min']))

reactive_X = list(Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km)
flocking_X = list(Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km)

reactive_X = list(metrics_hj_reactive['Mean maximum correction from optimal control degrees']) 
flocking_X = list(metrics_hj_flocking['Mean maximum correction from optimal control degrees'])

 #Compute the mean and standard deviation of beta values for each controller
reactive_mean = np.mean(reactive_X)
flocking_mean = np.mean(flocking_X)
reactive_std = np.std(reactive_X, ddof=1)
flocking_std = np.std(flocking_X, ddof=1)

# Compute the t-statistic and p-value using a two-sample t-test assuming unequal variances
t_stat, p_value = ttest_ind(reactive_X, flocking_X, equal_var=False) # perform Welch's t-test since the variances are unequal

# Print the results
print(f"Reactive controller mean: {reactive_mean:.3f}, std: {reactive_std:.3f}")
print(f"Flocking controller mean: {flocking_mean:.3f}, std: {flocking_std:.3f}")
print(f"Two-sample t-test results: t-statistic={t_stat:.3f}, p-value={p_value}")

if p_value < 0.05:
    print("The difference between the two controllers is statistically significant.")
else:
    print("There is no statistically significant difference between the two controllers.")


Reactive controller mean: 71.531, std: 26.335
Flocking controller mean: 128.896, std: 22.493
Two-sample t-test results: t-statistic=-52.744, p-value=0.0
The difference between the two controllers is statistically significant.


+ Collisions: no statistical significance ?

In [36]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= len(metrics_hj_naive)
N_success_reactive = len(metrics_hj_reactive[metrics_hj_reactive['Number_of_collision']==0])
N_success_flocking = len(metrics_hj_flocking[metrics_hj_flocking['Number_of_collision']==0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_success_flocking, N_success_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="smaller") #reject H0 if pval < 0.05 in flavor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

There is statistical significance between N_success_reactive and N_success_flocking.
0.012583503446428465


+ Communication: statistical significance ?

H0 is that there is no significant difference between the success rate of the reactive and the flocking methods in favor of the alternative hypothesis that the proportion of success for the flocking method is greater than the proportion of success for the flocking. Can reject H0 is p-value <= 0.05

In [35]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
N_total= len(metrics_hj_naive)
N_success_reactive = len(metrics_hj_reactive[metrics_hj_reactive['Isolated_platform_metric']==0])
N_success_flocking = len(metrics_hj_flocking[metrics_hj_flocking['Isolated_platform_metric']==0])

# count = np.array([N_success_flocking, N_success_reactive])
count = np.array([N_success_flocking, N_success_reactive])
nobs = np.array([N_total, N_total])
stat, pval = proportions_ztest(count, nobs, alternative="larger") #reject H0 if pval < 0.05 in flavor of first element of count smaller than second element
if pval > 0.05:
    print("There is no statistical significance between N_success_reactive and N_success_flocking.")
else:
    print("There is statistical significance between N_success_reactive and N_success_flocking.")
print(pval)

There is statistical significance between N_success_reactive and N_success_flocking.
9.184924473945803e-113


## Log Data in WandB

In [ ]:
import wandb

In [ ]:
# Log metrics in WandB
if wandb_log:
    os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

    wandb.init(
        # Set the project where this run will be logged
        project="Master_Thesis_Run_Summaries",
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=wandb_log_name,
        # Track hyperparameters and run metadata
        config={
            "Number of infeasible missions": len(non_feas_idx),
            "Percentage of infeasible missions": ratio_non_feas_miss,
            "Number of missions feasible": len(metrics_hj_naive),
            "Ratio failing missions with HJ naive in terms of connectivity and collisions": 
            len(metrics_hj_naive.index[(connect_objective_hj_naive==False)].tolist())/len(metrics_hj_naive),
        },
        entity="nhoischen",
    )
    wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
                commit=False)
    wandb.log({'Beta Index': fig_beta,
            'Time-Integral-over-Isolated-Platforms': fig_ipm,
            'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
            'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
                },
            commit=False)
    wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
            'Missions with collisions hj-flocking': fig_conf_flocking_coll,
            'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
            'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
            'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
            'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
            'Succesfull Missions hj-reactive': fig_react_success,
            'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
    wandb.finish()